In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

#获取网页内容
single_news_url = 'http://news.sina.com.cn/w/2018-07-09/doc-ihezpzwt6845530.shtml'
res = requests.get(single_news_url)
res.encoding = 'utf-8'

#用BeautifulSoup解析网页
soup = BeautifulSoup(res.text, 'html.parser')

In [2]:
#抓取新闻标题
main_title = soup.select('.main-title')[0].text


In [3]:
#抓取新闻时间与来源
news_time_str = soup.select('.date-source')[0].contents[1].text
news_time_tuple = datetime.strptime(news_time_str,'%Y年%m月%d日 %H:%M')#新闻时间，存储格式为元组
news_time_y_m_d_str = news_time_tuple.strftime('%Y-%m-%d')
news_source = soup.select('.date-source a')[0].text #新闻来源

In [4]:
#抓取文章正文内容
news_contents = ' '.join([p.text.strip() for p in soup.select('.article p')[:-1]])#文章正文内容，以空格作为每段的分割

In [5]:
#抓取责任编辑编辑
news_author = soup.select('.article p')[-1].text.strip('责任编辑：').strip(' ')

In [6]:
#抓取评论数和评论内容
import requests
import json
news_comments_url = 'http://comment5.news.sina.com.cn/page/info?version=1&format=json&channel=gj&\
newsid=comos-hezpzwt6845530&group=undefined&compress=0&ie=utf-8&oe=utf-8&page=1&page_size=3&t_size=3&\
h_size=3&thread=1' #在JS下面找，去掉loader部分
res_comments = requests.get(news_comments_url)
news_comments_json = json.loads(res_comments.text.strip('var data=')) #加载json数据
total_comments_number = news_comments_json['result']['count']['show']

In [7]:
total_comments_number

314

In [8]:
#第十六课 剖析新闻标志符news_id
## 方法一：使用python的spilt+strip方法
newsid1 = single_news_url.split('/')[-1].lstrip('doc-i').rstrip('.shtml')
## 方法二：使用正则表达式库re
import re
m = re.search('doc-i(.*).shtml',single_news_url)
newsid2 = m.group(1) #参数1表示只搜索通配符*中的内容，0表示搜索引号中''全部的内容


## 1.建立抓取评论数的函数

In [9]:
#第十七课 建立评论数抽取函数式
import re
import requests
news_comments_url = 'http://comment5.news.sina.com.cn/page/info?version=1&format=json&channel=gj&\
newsid=comos-{}&group=undefined&compress=0&ie=utf-8&oe=utf-8&page=1&page_size=3&t_size=3&\
h_size=3&thread=1' #把newsid部分用{}替换掉

def getCommentsCount(newsurl):
    '''获取单个新闻正文后面的评论数'''
    m = re.search('doc-i(.*).shtml',newsurl)#使用正则表达式搜索newsid
    newsid = m.group(1) #参数1表示只搜索通配符*中的内容，0表示搜索引号中''全部的内容
    res_comments = requests.get(news_comments_url.format(newsid)) #news_comments_url这个参数在函数外定义
    news_comments_json = json.loads(res_comments.text.strip('var data=')) #加载json数据
    total_comments_number = news_comments_json['result']['count']['show']
    return total_comments_number

In [10]:
#第十七课 建立评论数抽取函数式
##验证函数getCommentsCount
single_news_url = 'http://news.sina.com.cn/w/2018-07-09/doc-ihezpzwt6845530.shtml'
getCommentsCount(single_news_url)

314

## 2.建立抓取单个新闻的所有内容的函数

In [11]:
#第十八课 将抓取新闻内文整理成一个函数
import requests
from bs4 import BeautifulSoup
def getNewsContend(newsurl):
    '''传入单个新闻的url,返回该新闻的标题、时间与来源、文章正文内容、责任编辑、评论数'''
    result = {}
    res = requests.get(single_news_url)
    res.encoding = 'utf-8'

    #用BeautifulSoup解析网页
    soup = BeautifulSoup(res.text, 'html.parser')
    #抓取新闻标题
    result['title'] = soup.select('.main-title')[0].text
    news_time_str = soup.select('.date-source')[0].contents[1].text
    result['dt'] = datetime.strptime(news_time_str,'%Y年%m月%d日 %H:%M')#新闻时间，存储格式为元组
    result['source'] = soup.select('.date-source a')[0].text #新闻来源
    result['article'] = ' '.join([p.text.strip() for p in soup.select('.article p')[1:-1]])#文章正文内容，以空格作为每段的分割
    result['editor'] = soup.select('.article p')[-1].text.strip('责任编辑：').strip(' ')
    result['comments_number'] = getCommentsCount(newsurl) #还需要在调用这个函数前设置参数news_comments_url
    return result

In [12]:
#第十八课 将抓取新闻内文整理成一个函数
##验证getNewsContend函数
single_news_url = 'http://news.sina.com.cn/w/2018-07-09/doc-ihezpzwt6845530.shtml'
news_comments_url = 'http://comment5.news.sina.com.cn/page/info?version=1&format=json&channel=gj&\
newsid=comos-{}&group=undefined&compress=0&ie=utf-8&oe=utf-8&page=1&page_size=3&t_size=3&\
h_size=3&thread=1' #把newsid部分用{}替换掉
getNewsContend(single_news_url)

{'title': '美国说实话了：希望朝鲜走越南道路',
 'dt': datetime.datetime(2018, 7, 8, 23, 44),
 'source': '环球网',
 'article': '蓬佩奥表示，美国总统特朗普相信，朝鲜可以复制越南的发展道路，与美国关系正常化并获得经济繁荣。 路透社消息，正在越南访问的美国国务卿蓬佩奥8日敦促朝鲜遵循越南的先例，他表示，美国总统特朗普相信，朝鲜可以复制越南的发展道路，与美国关系正常化并获得经济繁荣。蓬佩奥两天前刚刚访问朝鲜，但平壤7日批评称，美国单方面对朝鲜提出了“强盗的无核化要求”。 据路透社报道，蓬佩奥今天在河内对经济界人士表示，他希望有朝一日美国可以与朝鲜建立与越南相当的伙伴关系，美越此前也曾是仇敌。他还表示，朝鲜如果可以复制越南道路，现在就需要把握时机。他还表示，越南崛起的关键在于与美国恢复了接触，两国在1985年起就开始就归还美军遗体展开合作。 此前，蓬佩奥刚刚访问了朝鲜，并称他的谈判对手、朝鲜劳动党中央委员会副委员长金英哲在会谈中展现出“善意”。但朝鲜外务省发言人7日通过朝中社发声，批评美国单方面对朝鲜提出了“强盗的无核化要求”。 环环：美国真是改不掉爱帮别人设计路的毛病。 ',
 'editor': '张迪',
 'comments_number': 314}

In [13]:
#第二十课 找到分页连接的参数
import requests
import json

#分页信息在参数page=3处
url = 'http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gjxw&level==1||=2&\
show_ext=1&show_all=1&show_num=22&tag=1&format=json&page=1&\
callback=newsloadercallback&_=1531207624777'

res = requests.get(url).text.lstrip('  newsloadercallback(').rstrip(');')
js = json.loads(res)

## 3.建立剖析一页新闻列表中每个新闻的连接函数

In [14]:
#第二十二课 建立剖析一页新闻列表中每个新闻的连接函数
import requests
import json

def getListLinks(page_news_url):
    '''获取一页新闻列表中每个新闻的连接'''
    news_link = []
    res = requests.get(url).text.lstrip('  newsloadercallback(').rstrip(');')
    js = json.loads(res)
    for new_link in js['result']['data']:
        news_link.append(new_link['url'])
    return news_link

In [15]:
##验证getListLinks函数
url = 'http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gjxw&level==1||=2&\
show_ext=1&show_all=1&show_num=22&tag=1&format=json&page=1&\
callback=newsloadercallback&_=1531207624777'
getListLinks(url)

['http://news.sina.com.cn/w/2018-07-10/doc-ihezpzwu8675327.shtml',
 'http://news.sina.com.cn/o/2018-07-10/doc-ihezpzwu8542137.shtml',
 'http://news.sina.com.cn/o/2018-07-10/doc-ihezpzwu8478791.shtml',
 'http://news.sina.com.cn/o/2018-07-10/doc-ihezpzwu8436387.shtml',
 'http://news.sina.com.cn/w/2018-07-10/doc-ihezpzwu8387578.shtml',
 'http://news.sina.com.cn/w/2018-07-10/doc-ihezpzwu8387622.shtml',
 'http://news.sina.com.cn/w/2018-07-10/doc-ihezpzwu8133792.shtml',
 'http://news.sina.com.cn/w/2018-07-10/doc-ihezpzwu7966287.shtml',
 'http://news.sina.com.cn/w/2018-07-10/doc-ihezpzwu7940010.shtml',
 'http://news.sina.com.cn/w/2018-07-10/doc-ihezpzwu7931467.shtml',
 'http://news.sina.com.cn/w/2018-07-10/doc-ihezpzwu7931048.shtml',
 'http://news.sina.com.cn/w/2018-07-10/doc-ihezpzwu7866828.shtml',
 'http://news.sina.com.cn/w/2018-07-10/doc-ihezpzwu7866726.shtml',
 'http://news.sina.com.cn/o/2018-07-10/doc-ihezpzwu7853490.shtml',
 'http://news.sina.com.cn/o/2018-07-10/doc-ihezpzwu7813640.sht

In [1]:
#第二十三课 使用for产生多页连接
url = 'http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gjxw&level==1||=2&\
show_ext=1&show_all=1&show_num=22&tag=1&format=json&page={}&\
callback=newsloadercallback&_=1531207624777'#这里用{}换page=1中的1

news_link_total = []
for i in range(1,6): #产生1-5页的所有新闻的链接
    fromat = url.format(i)
    news_link_total.extend(getListLinks(url.format(i))) #用extend方法而不是append方法

NameError: name 'getListLinks' is not defined

In [20]:
#第二十四课 整合所有，抓取每一条国际新闻的所有内容
import re
import requests
import json
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup

##1.产生每一条新闻的链接
url = 'http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gjxw&level==1||=2&\
show_ext=1&show_all=1&show_num=22&tag=1&format=json&page={}&\
callback=newsloadercallback&_=1531207624777'#这里用{}换page=1中的1
news_link_total = []
news_total = []
for i in range(1,6): #产生1-5页的所有新闻的链接
    fromat = url.format(i)
    news_link_total.extend(getListLinks(url.format(i))) #用extend方法而不是append方法
    
##2.使用for循环，获取每一条新闻链接里面的内容
for single_news_url in news_link_total:
    #这里一定要顶格！！！
    #否则在链接中会出现空格，导致读取不到json数据  
    news_comments_url =  'http://comment5.news.sina.com.cn/page/info?version=1&format=json&channel=gj\
&newsid=comos-{}&group=undefined&compress=0&ie=utf-8&oe=utf-8&page=1&page_size=3&\
t_size=3&h_size=3&thread=1'
    news_total.append(getNewsContend(single_news_url))#每一个链接返回一个字典

##3.使用pandas整理数据
df = pd.DataFrame(news_total)
df.to_excel('sina_world_news.xlsx')

,article,comments_number,dt,editor,source,title
0,[环球网快讯 记者 赵衍龙]泰国拯救人员已进入清莱岩洞，展开第三轮救援行动，路透社10日最新...,1,2018-07-10 17:23:00,张玉,环球网,泰国溶洞第3轮救援行动展开:第9人被救出 还剩4人
1,2012年印度新德里黑公交轮奸案9日终审，3名罪犯的上诉请求被驳回。 据《印度时报》10日报...,0,2018-07-10 16:44:00,余鹏飞,澎湃新闻,印2012年黑公交轮奸案终审 3名罪犯维持死刑判决
2,中国南海新闻网7月10日电 据越通社10日报道，越南国家历史博物馆9日下午在广义省平山县平顺...,0,2018-07-10 16:58:00,张玉,海外网,越南斥资逾千万发掘沉船 里面载有中国明朝瓷器
3,中新网7月10日电 综合报道，日前，日本遭遇大范围暴雨，灾情严重，已造成156人死亡，数十人...,0,2018-07-10 16:45:00,张玉,中国新闻网,日本暴雨致156死数十人失踪 1匹马躲屋顶幸免于难
4,[环球网报道 记者 赵怡蓁]瑞士资讯7月10日援引法新社报道称，荷兰安全研究人员日前发现，供...,0,2018-07-10 16:42:00,张玉,环球网,69国情报及军事人员行踪全暴露 只因他们干这件事
5,[环球网报道 记者 查希]特斯拉CEO马斯克7月10日发了一个新推特，他说：“刚刚从山洞回来...,0,2018-07-10 16:40:00,张玉,环球网,泰国溶洞救援马斯克来了又走 但留下救援潜艇(图)
6,[环球网综合报道]由于地震、海啸与台风等自然灾害发生频繁，日本平常就会为天灾做好准备，但这次...,0,2018-07-10 16:04:00,张玉,环球网,西日本地区此次为何受暴雨重创 外媒列五大原因
7,[环球网综合报道]泰国拯救人员已进入清莱岩洞，展开第三轮救援行动，希望把仍受困洞内的四名少年...,0,2018-07-10 15:38:00,张玉,环球网,泰岩洞救援总指挥：5个受困人今将在同个时候被救
8,[环球网报道 实习记者 杨璐]受台风和梅雨季节影响，西日本部分地区近日遭遇史无前例的大暴雨。...,0,2018-07-10 15:43:00,张玉,环球网,西日本遭遇史无前例暴雨 已致148人死亡59人失踪
9,海外网7月10日电 日本的暴雨灾情一度受到世界的广泛关注。截至目前，灾情造成的死亡人数已上升...,0,2018-07-10 14:56:00,张玉,海外网,西日本暴雨灾害已致141死 老妇谈灾情不禁落泪
